This script merges all individual files of trend predictors generated by "create_term_gpa_and_enrl_intensity_predictors.py" into one single table.

In [1]:
import pickle
import glob
import pandas as pd

In [2]:
enrl = glob.glob("~\\Box Sync\\Clickstream\\data\\full\\term_enrl_intensity_trend_values\*.p")
enrl_dict = {}
for f in enrl:
    d = pickle.load(open(f, "rb"))
    enrl_dict = {**enrl_dict, **d}

In [3]:
gpa = glob.glob("~\\Box Sync\\Clickstream\\data\\full\\term_gpa_trend_values\*.p")
gpa_dict = {}
for f in gpa:
    d = pickle.load(open(f, "rb"))
    gpa_dict = {**gpa_dict, **d}

In [4]:
enrl_df = pd.DataFrame.from_dict(enrl_dict, orient='index').reset_index()
enrl_df.loc[:,'vccsid'] = enrl_df['index'].apply(lambda x: x[0])
enrl_df.loc[:,'strm'] = enrl_df['index'].apply(lambda x: x[-1])
enrl_df = enrl_df.drop(['index'], axis=1)
enrl_df = enrl_df.rename(columns={0:'enrl_intensity_trend'})

In [5]:
gpa_df = pd.DataFrame.from_dict(gpa_dict, orient='index').reset_index()
gpa_df.loc[:,'vccsid'] = gpa_df['index'].apply(lambda x: x[0])
gpa_df.loc[:,'strm'] = gpa_df['index'].apply(lambda x: x[-1])
gpa_df = gpa_df.drop(['index'], axis=1)
gpa_df = gpa_df.rename(columns={0:'gpa_trend'})

In [6]:
trend_df = enrl_df.merge(gpa_df, how='inner', on=['vccsid', 'strm']).loc[:,['vccsid','strm','gpa_trend','enrl_intensity_trend']]

In [7]:
all_vccsid_strm = pd.read_stata("~\\Box Sync\\Clickstream\\data\\full\\all_vccsid_strm.dta")

In [8]:
trend_df = all_vccsid_strm.merge(trend_df, how='left', on=['vccsid', 'strm']).sort_values(['vccsid','strm']).loc[:,['vccsid','strm','gpa_trend','enrl_intensity_trend']]

In [9]:
trend_df.to_stata("~\\Box Sync\\Clickstream\\data\\full\\trend_predictors.dta", write_index=False)